In [1]:
import tensorflow as tf
import numpy as np
import matplotlib
import os
from sklearn.preprocessing import MinMaxScaler
from matplotlib import font_manager, rc

font_name = font_manager.FontProperties(fname="/usr/share/fonts/truetype/nanum/NanumGothic_Coding_Bold.ttf").get_name()
rc('font', family=font_name)

%matplotlib tk

tf.set_random_seed(777)  # reproducibility

if "DISPLAY" not in os.environ:
    # remove Travis CI Error
    matplotlib.use('Agg')

import matplotlib.pyplot as plt

In [2]:
# train Parameters
seq_length = 9
data_dim = 4
hidden_dim = 10
output_dim = 1
learning_rate = 0.01
iterations = 901

scaler = MinMaxScaler(feature_range=(0,1))
scaler2 = MinMaxScaler(feature_range=(0, 1))

# Open, High, Low, Volume, Close
xy = np.loadtxt('./v2data/ptmt/train1_v2_data.csv', delimiter=',')
x = scaler.fit_transform(xy[:, 0:-1])
y = scaler2.fit_transform(xy[:, [-1]])  # Close as label

print(xy[:, 0:-1])
print(xy[:, [-1]])

# build a dataset
dataX = []
dataY = []

for i in range(0, len(x) - seq_length):
    _x = x[i:i + seq_length]
    _y = y[i + seq_length]  # Next close price
    dataX.append(_x)
    dataY.append(_y)

xy2 = np.loadtxt('./sun_data/2018-04-12.csv',delimiter=',')
print(xy2)

x2 = scaler.fit_transform(xy2) # 데이터 없을때
# x2 = scaler.fit_transform(xy2[:, 0:-1]) # 데이터 있을때
# y2 = scaler2.fit_transform(xy2[:, [-1]]) #데이터 있을때

dataX2 = []
# dataY2 = [] #데이터 있을때

for i in range(0, len(x2) - seq_length):
    _x2 = x2[i:i+seq_length]
#     _y2 = y2[i+seq_length] #데이터 있을때
    dataX2.append(_x2)
#     dataY2.append(_y2) #데이터 있을때
# train/test split

train_size = len(dataX)
test_size = len(dataX2)

trainX, testX = np.array(dataX[0:train_size]),np.array(dataX2[0:test_size])
trainY = np.array(dataY[0:train_size])

[[ 57.  -2.  83.  23.]
 [ 58.  -2.  83.  23.]
 [ 58.  -2.  83.  23.]
 ...
 [268. -23. 322. -14.]
 [268. -23. 322. -14.]
 [268. -23. 322. -14.]]
[[6.56707e+01]
 [7.64514e+01]
 [8.85869e+01]
 ...
 [2.38816e-02]
 [2.47933e-02]
 [2.35017e-02]]
[[ 68. -12.  20.   8.]
 [ 69. -12.  20.   8.]
 [ 69. -12.  20.   8.]
 ...
 [290. -11.  20.   8.]
 [290. -11.  20.   8.]
 [290. -11.  20.   8.]]


In [3]:
# input place holders
X = tf.placeholder(tf.float32, [None, seq_length, data_dim])
Y = tf.placeholder(tf.float32, [None, 1])

# build a LSTM network
cell = tf.contrib.rnn.BasicLSTMCell(
    num_units=hidden_dim, state_is_tuple=True, activation=tf.tanh)
outputs, _states = tf.nn.dynamic_rnn(cell, X, dtype=tf.float32)
Y_pred = tf.contrib.layers.fully_connected(
    outputs[:, -1], output_dim, activation_fn=None)  # We use the last cell's output

# cost/loss
loss = tf.reduce_sum(tf.square(Y_pred - Y))  # sum of the squares
# optimizer
optimizer = tf.train.AdamOptimizer(learning_rate)
train = optimizer.minimize(loss)

Instructions for updating:
Use the retry module or similar alternatives.


In [4]:
with tf.Session() as sess:
    init = tf.global_variables_initializer()
    sess.run(init)

    # Training step
    for i in range(iterations):
        _, step_loss = sess.run([train, loss], feed_dict={
                                X: trainX, Y: trainY})
        print("[step: {}] loss: {}".format(i, step_loss))
    # Test step
    test_predict = sess.run(Y_pred, feed_dict={X: testX})
    
    test_predict = scaler2.inverse_transform(test_predict)
#     dataY2 = scaler2.inverse_transform(dataY2) # 데이터 있을때

[step: 0] loss: 2258.882080078125
[step: 1] loss: 1660.694580078125
[step: 2] loss: 1197.8079833984375
[step: 3] loss: 852.0170288085938
[step: 4] loss: 629.59912109375
[step: 5] loss: 541.6940307617188
[step: 6] loss: 565.2955322265625
[step: 7] loss: 620.8690795898438
[step: 8] loss: 635.406005859375
[step: 9] loss: 596.9840698242188
[step: 10] loss: 529.6331176757812
[step: 11] loss: 460.34783935546875
[step: 12] loss: 406.0227966308594
[step: 13] loss: 372.6711120605469
[step: 14] loss: 358.565673828125
[step: 15] loss: 357.4196472167969
[step: 16] loss: 360.7814025878906
[step: 17] loss: 360.1778259277344
[step: 18] loss: 349.28369140625
[step: 19] loss: 325.50494384765625
[step: 20] loss: 290.63018798828125
[step: 21] loss: 250.84361267089844
[step: 22] loss: 216.0191192626953
[step: 23] loss: 196.56802368164062
[step: 24] loss: 195.6203155517578
[step: 25] loss: 201.45733642578125
[step: 26] loss: 194.50184631347656
[step: 27] loss: 167.23782348632812
[step: 28] loss: 131.529464

[step: 233] loss: 8.544663429260254
[step: 234] loss: 8.527522087097168
[step: 235] loss: 8.510476112365723
[step: 236] loss: 8.493517875671387
[step: 237] loss: 8.476648330688477
[step: 238] loss: 8.459870338439941
[step: 239] loss: 8.443172454833984
[step: 240] loss: 8.426563262939453
[step: 241] loss: 8.4100341796875
[step: 242] loss: 8.393579483032227
[step: 243] loss: 8.37721061706543
[step: 244] loss: 8.36091423034668
[step: 245] loss: 8.34469223022461
[step: 246] loss: 8.32854175567627
[step: 247] loss: 8.31246566772461
[step: 248] loss: 8.296462059020996
[step: 249] loss: 8.280526161193848
[step: 250] loss: 8.264657020568848
[step: 251] loss: 8.248848915100098
[step: 252] loss: 8.233113288879395
[step: 253] loss: 8.217436790466309
[step: 254] loss: 8.201824188232422
[step: 255] loss: 8.186272621154785
[step: 256] loss: 8.170782089233398
[step: 257] loss: 8.15534782409668
[step: 258] loss: 8.139974594116211
[step: 259] loss: 8.124652862548828
[step: 260] loss: 8.109389305114746


[step: 461] loss: 5.520538330078125
[step: 462] loss: 5.508061408996582
[step: 463] loss: 5.495572566986084
[step: 464] loss: 5.48307466506958
[step: 465] loss: 5.47056245803833
[step: 466] loss: 5.458037853240967
[step: 467] loss: 5.445502281188965
[step: 468] loss: 5.432955265045166
[step: 469] loss: 5.420393943786621
[step: 470] loss: 5.4078240394592285
[step: 471] loss: 5.39523983001709
[step: 472] loss: 5.382641792297363
[step: 473] loss: 5.370030879974365
[step: 474] loss: 5.357409477233887
[step: 475] loss: 5.344771862030029
[step: 476] loss: 5.332121849060059
[step: 477] loss: 5.319457530975342
[step: 478] loss: 5.306780815124512
[step: 479] loss: 5.294088363647461
[step: 480] loss: 5.281383037567139
[step: 481] loss: 5.26866340637207
[step: 482] loss: 5.255929470062256
[step: 483] loss: 5.243181228637695
[step: 484] loss: 5.230417251586914
[step: 485] loss: 5.217638969421387
[step: 486] loss: 5.204845428466797
[step: 487] loss: 5.192036151885986
[step: 488] loss: 5.17921304702

[step: 689] loss: 2.8098766803741455
[step: 690] loss: 2.803746223449707
[step: 691] loss: 2.797689437866211
[step: 692] loss: 2.791706085205078
[step: 693] loss: 2.785796880722046
[step: 694] loss: 2.779960870742798
[step: 695] loss: 2.7741942405700684
[step: 696] loss: 2.768501043319702
[step: 697] loss: 2.7628772258758545
[step: 698] loss: 2.757323741912842
[step: 699] loss: 2.751838445663452
[step: 700] loss: 2.746422052383423
[step: 701] loss: 2.7410736083984375
[step: 702] loss: 2.735793352127075
[step: 703] loss: 2.730576515197754
[step: 704] loss: 2.7254269123077393
[step: 705] loss: 2.7203421592712402
[step: 706] loss: 2.7153210639953613
[step: 707] loss: 2.7103629112243652
[step: 708] loss: 2.7054684162139893
[step: 709] loss: 2.7006373405456543
[step: 710] loss: 2.6958649158477783
[step: 711] loss: 2.691153049468994
[step: 712] loss: 2.686501979827881
[step: 713] loss: 2.681910753250122
[step: 714] loss: 2.6773767471313477
[step: 715] loss: 2.672900676727295
[step: 716] loss

In [5]:
# Plot predictions
plt.title('2018-04-09 predict')
# plt.plot(dataY2, label='row data') # 데이터 있을때
plt.plot(test_predict, label='predict')
plt.xlabel("Time Period")
plt.ylabel("photometric")
# plt.ylim(0, 140000)
plt.xticks(np.arange(0, 901, step = 56), ['5', '6', '7', '8', '9', '10', '11', '12', '1', '2', '3', '4', '5', '6', '7', '8']) # v2 모델
plt.yticks(np.arange(0, 140000, step=10000))
plt.grid(True)
plt.legend(loc='upper right')
plt.show()